In [9]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 결과 재현성을 위해 랜덤 시드 고정
torch.manual_seed(777)
# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
learning_rate = 0.001
training_epochs = 15
batch_size = 50
mnist_train = dsets.MNIST(root = 'train/', # 다운로드 경로 지정
	train = True, # True를 지정하면 훈련 데이터로 다운로드
	transform = transforms.ToTensor(), # 텐서로 변환
	download = True)
mnist_test = dsets.MNIST(root = 'test/', # 다운로드 경로 지정
    train = False, # False를 지정하면 테스트 데이터로 다운로드
    transform = transforms.ToTensor(), # 텐서로 변환
    download = True)
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
    batch_size = batch_size,
    shuffle = True,
    drop_last = True)

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
    	# 첫번째층
    	# ImgIn shape = (? , 28, 28, 1)
    	# Conv -> (? , 28, 28, 32)
    	# Pool -> (? , 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
    		torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
    		torch.nn.ReLU(),
    		torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    	# 두번째층
    	# ImgIn shape = (? , 14, 14, 32)
    	# Conv ->(? , 14, 14, 64)
    	# Pool ->(? , 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
    		torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
    		torch.nn.ReLU(),
    		torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
    	# 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
    	# 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out
# CNN 모델 정의
model = CNN().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))
for epoch in range(training_epochs) :
    avg_cost = 0
    for X, Y in data_loader : # 미니 배치 단위로 꺼내옴.X는 미니 배치, Y는 레이블.
        # image is already size of(28x28), no reshape
        # label is not one - hot encoded
        X = X.to(device)
        Y = Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad() :
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

총 배치의 수 : 1200
[Epoch:    1] cost = 0.169384092
[Epoch:    2] cost = 0.0523789786
[Epoch:    3] cost = 0.0385489017
[Epoch:    4] cost = 0.0299722925
[Epoch:    5] cost = 0.0236043707
[Epoch:    6] cost = 0.0196063705
[Epoch:    7] cost = 0.0158073213
[Epoch:    8] cost = 0.0137237655
[Epoch:    9] cost = 0.0101975333
[Epoch:   10] cost = 0.00884883106
[Epoch:   11] cost = 0.00725776609
[Epoch:   12] cost = 0.00657538697
[Epoch:   13] cost = 0.0050253165
[Epoch:   14] cost = 0.00526297977
[Epoch:   15] cost = 0.00389165478


C:\Users\kong7\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\kong7\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9811000227928162


In [188]:
import os
from PIL import Image
import numpy as np
import cv2
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from skimage.morphology import thin, skeletonize
from skimage.filters import threshold_otsu
from skimage.measure import find_contours
from skimage.transform import rotate
from scipy.ndimage import shift
from skimage.transform import warp

np.random.seed(42) # random값 고정

class CustomImageDataset:
    def read_data_set(self):
        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("L")  # Convert to grayscale
        image = np.array(image)

        # 이진화 적용
        thresh = threshold_otsu(image)
        binary = image > thresh

        # 숫자 영역 탐지 및 잘라내기
        contours = find_contours(binary, 0.8)
        if len(contours) > 0:
            # 가장 큰 윤곽선 선택
            contour = max(contours, key=lambda c: c.shape[0])
            # 윤곽선의 좌표 추출
            y, x = contour.T
            # 윤곽선의 경계 좌표 계산
            x_min, x_max = x.min(), x.max()
            y_min, y_max = y.min(), y.max()
            # 숫자 영역 잘라내기
            cropped = binary[int(y_min):int(y_max), int(x_min):int(x_max)]
        else:
            # 윤곽선이 없는 경우 이진화된 이미지 그대로 사용
            cropped = binary
            
        # 크기 정규화
        cropped = np.array(Image.fromarray(cropped).resize((32, 32)))

        # 모폴로지 연산 적용
        thinned = thin(cropped)
        skeleton = skeletonize(thinned)

        # 특성 추출
        features = self.extract_features(skeleton)

        return features, self.labels[index]

    def __len__(self):
        return self.length

    def extract_features(self, image):
        # 이미지를 1차원 배열로 변환
        features = image.flatten()

        return features


def augment_data(X, y):
    augmented_X = []
    augmented_y = []

    for image, label in zip(X, y):
        # 원본 이미지 추가
        augmented_X.append(image)
        augmented_y.append(label)

        # 회전 변환 추가
        for angle in [-5, 5]:
            rotated_image = rotate(image.reshape(32, 32), angle).flatten()
            augmented_X.append(rotated_image)
            augmented_y.append(label)

        # 이동 변환 추가
        for shift_x, shift_y in [(2, 0), (-2, 0), (0, 2), (0, -2)]:
            shifted_image = shift(image.reshape(32, 32), (shift_x, shift_y), mode='constant').flatten()
            augmented_X.append(shifted_image)
            augmented_y.append(label)

        image_reshaped = image.reshape(32, 32)
        mapping = np.indices((32, 32)).transpose(1, 2, 0).astype(np.float64)
        mapping += np.random.rand(*mapping.shape) * 7
        mapping = mapping.transpose(2, 0, 1)  # Transpose to (2, 32, 32)
        curved_image = warp(image_reshaped, mapping, mode='wrap').flatten()
        augmented_X.append(curved_image)
        augmented_y.append(label)
        
    return np.array(augmented_X), np.array(augmented_y)

train_data_set = CustomImageDataset(data_set_path="./train")
test_data_set = CustomImageDataset(data_set_path="./test")

if not (train_data_set.num_classes == test_data_set.num_classes):
    print("error: Numbers of class in training set and test set are not equal")
    exit()

X_train, y_train = zip(*[train_data_set[i] for i in range(len(train_data_set))])
X_test, y_test = zip(*[test_data_set[i] for i in range(len(test_data_set))])

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# 데이터 증강 적용
X_train, y_train = augment_data(X_train, y_train)

# Create a Gaussian Naive Bayes classifier
clf = GaussianNB()

# Train the classifier
clf.fit(X_train, y_train)

# Predict the labels of the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy * 100, "%")

Accuracy: 85.0 %


In [42]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms


class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return {'image': image, 'label': self.labels[index]}

    def __len__(self):
        return self.length


class CustomConvNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomConvNet, self).__init__()

        self.layer1 = self.conv_module(3, 16)
        self.layer2 = self.conv_module(16, 32)
        self.layer3 = self.conv_module(32, 64)
        self.layer4 = self.conv_module(64, 128)
        self.layer5 = self.conv_module(128, 256)
        self.gap = self.global_avg_pool(256, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.gap(out)
        out = out.view(-1, num_classes)

        return out

    def conv_module(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

    def global_avg_pool(self, in_num, out_num):
        return nn.Sequential(
            nn.Conv2d(in_num, out_num, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_num),
            nn.LeakyReLU(),
            nn.AdaptiveAvgPool2d((1, 1)))


hyper_param_epoch = 20
hyper_param_batch = 8
hyper_param_learning_rate = 0.001

transforms_train = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])

transforms_test = transforms.Compose([transforms.Resize((128, 128)),
                                      transforms.ToTensor()])

train_data_set = CustomImageDataset(data_set_path="./train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)

test_data_set = CustomImageDataset(data_set_path="./test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)

if not (train_data_set.num_classes == test_data_set.num_classes):
    print("error: Numbers of class in training set and test set are not equal")
    exit()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_classes = train_data_set.num_classes
custom_model = CustomConvNet(num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

for e in range(hyper_param_epoch):
    for i_batch, item in enumerate(train_loader):
        images = item['image'].to(device)
        labels = item['label'].to(device)

        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i_batch + 1) % hyper_param_batch == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'
                  .format(e + 1, hyper_param_epoch, loss.item()))

# Test the model
custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        images = item['image'].to(device)
        labels = item['label'].to(device)
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))

Epoch [1/30], Loss: 1.6781
Epoch [2/30], Loss: 1.2937
Epoch [3/30], Loss: 1.3520
Epoch [4/30], Loss: 1.6942
Epoch [5/30], Loss: 1.1172
Epoch [6/30], Loss: 1.0233
Epoch [7/30], Loss: 1.0070
Epoch [8/30], Loss: 1.0377
Epoch [9/30], Loss: 1.1071
Epoch [10/30], Loss: 1.1089
Epoch [11/30], Loss: 0.7011
Epoch [12/30], Loss: 0.7619
Epoch [13/30], Loss: 0.7036
Epoch [14/30], Loss: 0.5623
Epoch [15/30], Loss: 0.8458
Epoch [16/30], Loss: 0.6536
Epoch [17/30], Loss: 0.8533
Epoch [18/30], Loss: 0.6321
Epoch [19/30], Loss: 0.6169
Epoch [20/30], Loss: 0.8469
Epoch [21/30], Loss: 0.4531
Epoch [22/30], Loss: 0.7140
Epoch [23/30], Loss: 0.5836
Epoch [24/30], Loss: 0.4722
Epoch [25/30], Loss: 0.6766
Epoch [26/30], Loss: 0.6483
Epoch [27/30], Loss: 0.4565
Epoch [28/30], Loss: 0.6136
Epoch [29/30], Loss: 0.5193
Epoch [30/30], Loss: 0.4712
Test Accuracy of the model on the 20 test images: 100.0 %


In [37]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def read_data_set(self):
        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("L")
        image = image.resize((28, 28))
        image = np.array(image).astype(np.float32) / 255.0
        image = torch.from_numpy(image.reshape(-1)).float()
        label = torch.tensor(self.labels[index]).long()
        return image, label

    def __len__(self):
        return self.length

class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

train_data_set = CustomImageDataset(data_set_path="./train")
test_data_set = CustomImageDataset(data_set_path="./test")

batch_size = 64
train_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_set, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 28 * 28
hidden_size = 128
num_classes = train_data_set.num_classes
model = Net(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000  # 에포크 수를 200으로 증가
best_accuracy = 0.0

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_model.pth")  # 최고 성능 모델 저장

print(f"Best Test Accuracy: {best_accuracy:.2f}%")

Epoch [1/1000], Test Accuracy: 25.00%
Epoch [2/1000], Test Accuracy: 25.00%
Epoch [3/1000], Test Accuracy: 25.00%
Epoch [4/1000], Test Accuracy: 25.00%
Epoch [5/1000], Test Accuracy: 30.00%
Epoch [6/1000], Test Accuracy: 25.00%
Epoch [7/1000], Test Accuracy: 25.00%
Epoch [8/1000], Test Accuracy: 25.00%
Epoch [9/1000], Test Accuracy: 25.00%
Epoch [10/1000], Test Accuracy: 25.00%
Epoch [11/1000], Test Accuracy: 25.00%
Epoch [12/1000], Test Accuracy: 25.00%
Epoch [13/1000], Test Accuracy: 35.00%
Epoch [14/1000], Test Accuracy: 50.00%
Epoch [15/1000], Test Accuracy: 25.00%
Epoch [16/1000], Test Accuracy: 25.00%
Epoch [17/1000], Test Accuracy: 25.00%
Epoch [18/1000], Test Accuracy: 35.00%
Epoch [19/1000], Test Accuracy: 35.00%
Epoch [20/1000], Test Accuracy: 45.00%
Epoch [21/1000], Test Accuracy: 25.00%
Epoch [22/1000], Test Accuracy: 25.00%
Epoch [23/1000], Test Accuracy: 20.00%
Epoch [24/1000], Test Accuracy: 35.00%
Epoch [25/1000], Test Accuracy: 40.00%
Epoch [26/1000], Test Accuracy: 25

In [41]:
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class CustomImageDataset(Dataset):
    def read_data_set(self):
        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("L")
        image = image.resize((28, 28))
        image = np.array(image).astype(np.float32) / 255.0
        image = torch.from_numpy(image).unsqueeze(0)
        label = torch.tensor(self.labels[index]).long()
        return image, label

    def __len__(self):
        return self.length

class AttentionCNN(nn.Module):
    def __init__(self, num_classes):
        super(AttentionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.attention = nn.Linear(64*7*7, 64)  # 수정: 입력 크기를 64*7*7로 변경
        self.fc = nn.Linear(64, num_classes)  # 수정: 입력 크기를 64로 변경
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        
        # Attention Mechanism
        batch_size, channels, height, width = out.size()
        attention_weights = self.attention(out.view(batch_size, -1))  # 수정: 입력을 (batch_size, 64*7*7)로 변경
        attention_weights = torch.softmax(attention_weights, dim=-1)
        attention_weights = attention_weights.unsqueeze(-1).unsqueeze(-1)  # 수정: 어텐션 가중치의 크기를 (batch_size, 64, 1, 1)로 변경
        out = out * attention_weights  # 수정: 브로드캐스팅을 사용하여 어텐션 가중치 적용
        
        out = torch.sum(out, dim=[2, 3])  # 수정: 공간 차원(height, width)에 대해 합을 계산하여 (batch_size, 64)로 변경
        out = self.fc(out)
        return out

def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = 100. * correct / len(test_loader.dataset)
    return accuracy

train_data_set = CustomImageDataset(data_set_path="./train")
test_data_set = CustomImageDataset(data_set_path="./test")

batch_size = 64
train_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_set, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = train_data_set.num_classes
model = AttentionCNN(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 500
best_accuracy = 0.0

for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, criterion, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Test Accuracy: {accuracy:.2f}%")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_model.pth")

print(f"Best Test Accuracy: {best_accuracy:.2f}%")

Epoch [1/500], Test Accuracy: 25.00%
Epoch [2/500], Test Accuracy: 25.00%
Epoch [3/500], Test Accuracy: 25.00%
Epoch [4/500], Test Accuracy: 25.00%
Epoch [5/500], Test Accuracy: 25.00%
Epoch [6/500], Test Accuracy: 20.00%
Epoch [7/500], Test Accuracy: 25.00%
Epoch [8/500], Test Accuracy: 35.00%
Epoch [9/500], Test Accuracy: 25.00%
Epoch [10/500], Test Accuracy: 25.00%
Epoch [11/500], Test Accuracy: 25.00%
Epoch [12/500], Test Accuracy: 30.00%
Epoch [13/500], Test Accuracy: 30.00%
Epoch [14/500], Test Accuracy: 20.00%
Epoch [15/500], Test Accuracy: 25.00%
Epoch [16/500], Test Accuracy: 40.00%
Epoch [17/500], Test Accuracy: 35.00%
Epoch [18/500], Test Accuracy: 25.00%
Epoch [19/500], Test Accuracy: 45.00%
Epoch [20/500], Test Accuracy: 40.00%
Epoch [21/500], Test Accuracy: 30.00%
Epoch [22/500], Test Accuracy: 40.00%
Epoch [23/500], Test Accuracy: 40.00%
Epoch [24/500], Test Accuracy: 40.00%
Epoch [25/500], Test Accuracy: 25.00%
Epoch [26/500], Test Accuracy: 45.00%
Epoch [27/500], Test 